In [138]:
from math import exp,log,sqrt
from scipy.stats import norm
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
from scipy.optimize import fsolve
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from risklib.option import gbsm
from risklib.riskstats import VaR,ES

## Problem1

In [2]:
# The closed form greek formulas
def cf_greeks(underlying,strike,ttm,rf,b,ivol,call=True):
    d1 = (log(underlying/strike)+(b+ivol**2/2)*ttm)/(ivol*sqrt(ttm))
    d2 = d1 - ivol*sqrt(ttm)
    if call:
        delta = exp((b-rf)*ttm) * norm(0,1).cdf(d1)
        theta = - underlying*exp((b-rf)*ttm)*norm(0,1).pdf(d1)*ivol/(2*sqrt(ttm)) \
        - (b-rf)*underlying*exp((b-rf)*ttm)*norm(0,1).cdf(d1)-rf*strike*exp(-rf*ttm)*norm(0,1).cdf(d2)
        #rho for rf != b
        rho = ttm*strike*exp(-rf*ttm)*norm(0,1).cdf(d2)-ttm*underlying*norm(0,1).cdf(d1)*exp((b-rf)*ttm)
        carry_rho = ttm*underlying*exp((b-rf)*ttm)*norm(0,1).cdf(d1)
    else:
        delta = exp((b-rf)*ttm) * (norm(0,1).cdf(d1)-1)
        theta = - underlying*exp((b-rf)*ttm)*norm(0,1).pdf(d1)*ivol/(2*sqrt(ttm)) \
        + (b-rf)*underlying*exp((b-rf)*ttm)*norm(0,1).cdf(-d1)+ rf*strike*exp(-rf*ttm)*norm(0,1).cdf(-d2)
        rho = -ttm*strike*exp(-rf*ttm)*norm(0,1).cdf(-d2)+ttm*underlying*norm(0,1).cdf(-d1)*exp((b-rf)*ttm)
        carry_rho = -ttm*underlying*exp((b-rf)*ttm)*norm(0,1).cdf(-d1)
    gamma = norm(0,1).pdf(d1)* exp((b-rf)*ttm)/ (underlying*ivol*sqrt(ttm))
    vega = underlying * exp((b-rf)*ttm) * norm(0,1).pdf(d1) * sqrt(ttm)
    # return greeks
    return delta,gamma,vega,theta,rho,carry_rho

In [2]:
# Inputs
current_price = 165
strike = 165
curr_date = datetime(2022,3,13)
expiration = datetime(2022,4,15)
rf = 0.0425
q=0.0053
ivol = 0.2
ttm = (expiration-curr_date).days/365

In [4]:
# print out the greeks using closed form formula for call:
delta,gamma,vega,theta,rho,carry_rho = cf_greeks(current_price,strike,ttm,rf,rf-q,ivol,True)
print("Greeks for call:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))

Greeks for call:
Deta:0.5340091224850149
gamma:0.040037930803986446
Vega: 19.710179716477544
theta: -24.898522316969515
rho: -0.38265959627823776
carry_rho: 7.966245676523029


In [5]:
# print out the greeks using closed form formula for put:
delta,gamma,vega,theta,rho,carry_rho = cf_greeks(current_price,strike,ttm,rf,rf-q,ivol,False)
print("Greeks for Put:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))

Greeks for Put:
Deta:-0.4655118142202754
gamma:0.040037930803986446
Vega: 19.710179716477544
theta: -18.786996965277233
rho: -0.33259498982809
carry_rho: -6.944415968299725


In [6]:
# The finite difference method
def fd_greeks(d_s,d_v,d_t,d_r,d_b,current_price,strike,ttm,rf,b,ivol,call):
    f_diff = (gbsm(current_price+d_s,strike,ttm,rf,b,ivol,call=call)-gbsm(current_price,strike,ttm,rf,b,ivol,call=call))/d_s
    b_diff = (gbsm(current_price,strike,ttm,rf,b,ivol,call=call)-gbsm(current_price-d_s,strike,ttm,rf,b,ivol,call=call))/d_s
    delta = f_diff
    gamma = (f_diff-b_diff)/d_s
    vega = (gbsm(current_price,strike,ttm,rf,b,ivol+d_v,call=call)-gbsm(current_price,strike,ttm,rf,b,ivol,call=call))/d_v
    theta = -(gbsm(current_price,strike,ttm+d_t,rf,b,ivol,call=call)-gbsm(current_price,strike,ttm,rf,b,ivol,call=call))/d_t
    rho = (gbsm(current_price,strike,ttm,rf+d_r,b,ivol,call=call)-gbsm(current_price,strike,ttm,rf,b,ivol,call=call))/d_r
    carry_rho = (gbsm(current_price,strike,ttm,rf,b+d_b,ivol,call=call)-gbsm(current_price,strike,ttm,rf,b,ivol,call=call))/d_b
    return delta,gamma,vega,theta,rho,carry_rho

In [7]:
# Print out finite difference greeks for call
delta,gamma,vega,theta,rho,carry_rho=fd_greeks(0.2,0.01,0.01,0.01,0.007,current_price,strike,ttm,rf,rf-q,ivol,True)
print("Greeks for call:")
print("Delta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))

Greeks for call:
Deta:0.5380088534424488
gamma:0.04003661620579635
Vega: 19.71119469377811
theta: -24.322438934413526
rho: -0.38248666529199227
carry_rho: 7.999955781329504


In [8]:
# Print out finite difference greeks for put
delta,gamma,vega,theta,rho,carry_rho=fd_greeks(0.2,0.01,0.01,0.01,0.007,current_price,strike,ttm,rf,rf-q,ivol,False)
print("Greeks for Put:")
print("Delta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("carry_rho: {}".format(carry_rho))

Greeks for Put:
Deta:-0.4615120832627184
gamma:0.04003661620615162
Vega: 19.71119469377811
theta: -18.212374651071173
rho: -0.33244468396702587
carry_rho: -6.915425164275948


In [233]:
# binomial tree without dividend
def bt_american(underlying,strike,ttm,rf,b,ivol,N,call):
    dt = ttm/N
    u = exp(ivol*sqrt(dt))
    d = 1/u
    pu = (exp(b*dt)-d)/(u-d)
    pd = 1.0-pu
    df = exp(-rf*dt)
    if call:
        z=1
    else:
        z=-1
    # calculate the number of nodes
    def nNode(n):
        return int((n+1)*(n+2)/2)
    # Calculate the index
    def idx(i,j):
        return nNode(j-1)+i
    nNodes = nNode(N)
    optionvalues = np.zeros(nNodes)
    for j in range(N,-1,-1):
        for i in range(j,-1,-1):
            index = idx(i,j)
            price = underlying*u**i*d**(j-i)
            optionvalues[index]=max(0,z*(price-strike))
            if j<N:
               optionvalues[index] = max(optionvalues[index],df*(pu*optionvalues[idx(i+1,j+1)]+pd*optionvalues[idx(i,j+1)]))
            # print(i,j,optionvalues[index])
    return optionvalues[0]

In [234]:
# binomial tree with dividend
def bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call):
    # No dividends
    if len(divamts)==0 or len(divtimes)==0:
        return bt_american(underlying,strike,ttm,rf,rf,ivol,N,call)
    # First div outside grid
    if divtimes[0]>N:
        return bt_american(underlying,strike,ttm,rf,rf,ivol,N,call)
    dt = ttm/N
    u = exp(ivol*sqrt(dt))
    d = 1/u
    pu = (exp(rf*dt)-d)/(u-d)
    pd = 1.0-pu
    df = exp(-rf*dt)
    if call:
        z=1
    else:
        z=-1
    # calculate the number of nodes
    def nNode(n):
        return int((n+1)*(n+2)/2)
    # Calculate the index
    def idx(i,j):
        return nNode(j-1)+i
    nDiv = len(divtimes)
    nNodes = nNode(divtimes[0])
    optionvalues = np.zeros(nNodes)
    for j in range(divtimes[0],-1,-1):
        for i in range(j,-1,-1):
            index = idx(i,j)
            price = underlying*u**i*d**(j-i)
            if j < divtimes[0]:
                # Times before dividend, backward method
                optionvalues[index]=max(0,z*(price-strike))
                optionvalues[index] = max(optionvalues[index],df*(pu*optionvalues[idx(i+1,j+1)]+pd*optionvalues[idx(i,j+1)]))
            else:
                valnoex = bt_american_div(price-divamts[0],strike,ttm-divtimes[0]*dt,rf,divamts[1:nDiv-1],divtimes[1:nDiv-1]-divtimes[0],ivol,N-divtimes[0],call)
                valex = max(0,z*(price-strike))
                optionvalues[index] = max(valnoex,valex)
                # print("new",i,j,optionvalues[index])
    return optionvalues[0]

In [235]:
# Inputs
underlying = 165
strike = 165
curr_date = datetime(2022,3,13)
expiration = datetime(2022,4,15)
div_date = datetime(2022,4,11)
rf = 0.0425
q=0.0053
ivol = 0.2
ttm = (expiration-curr_date).days/365
N=200
divtimes = int((div_date-curr_date).days/(expiration-curr_date).days *N)
div=0.88

In [236]:
# Values without dividend
call_nodiv = bt_american(underlying,strike,ttm,rf,rf,ivol,N,True)
put_nodiv = bt_american(underlying,strike,ttm,rf,rf,ivol,N,False)
print("Call option without dividend:",call_nodiv)
print("Put option without dividend:",put_nodiv)
# Values with dividend
call_div = bt_american_div(underlying,strike,ttm,rf,np.array([div]),np.array([divtimes]),ivol,N,True)
put_div = bt_american_div(underlying,strike,ttm,rf,np.array([div]),np.array([divtimes]),ivol,N,False)
print("Call option with dividend:",call_div)
print("Put option with dividend:",put_div)

Call option without dividend: 4.2698585632362684
Put option without dividend: 3.684138176821656
Call option with dividend: 4.112836095267345
Put option with dividend: 4.1105345298444895


In [237]:
# Calculate greeks for bt
def fd_greeks_bt(d_s,d_v,d_t,d_r,d_d,underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call):
    f_diff = (bt_american_div(underlying+d_s,strike,ttm,rf,divamts,divtimes,ivol,N,call)-bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call))/d_s
    b_diff = (bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call)-bt_american_div(underlying-d_s,strike,ttm,rf,divamts,divtimes,ivol,N,call))/d_s
    delta = f_diff
    gamma = (f_diff-b_diff)/d_s
    vega = (bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol+d_v,N,call)- bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call))/d_v
    theta = -(bt_american_div(underlying,strike,ttm+d_t,rf,divamts,divtimes,ivol,N,call)-bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call))/d_t
    rho = (bt_american_div(underlying,strike,ttm,rf+d_r,divamts,divtimes,ivol,N,call)-bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call))/d_r
    # The derivative respect to dividend
    ddiv = (bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,ivol,N,call)-bt_american_div(underlying,strike,ttm,rf,divamts-d_d,divtimes,ivol,N,call))/d_r
    return delta,gamma,vega,theta,rho,ddiv

In [242]:
# Greeks without dividend
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,underlying,strike,ttm,rf,np.array([]),np.array([]),ivol,N,True)
print("Greeks for call:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))

Greeks for call:
Deta:0.5654346292413059
gamma:0.28050135221535655
Vega: 19.68233756779254
theta: -24.781072878797517
rho: 7.671558546741863


In [243]:
# Greeks without dividend
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,underlying,strike,ttm,rf,np.array([]),np.array([]),ivol,N,False)
print("Greeks for Put:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))

Greeks for Put:
Deta:-0.4476915654878666
gamma:0.24295249234087546
Vega: 19.645185811427222
theta: -18.386850803746224
rho: -5.813164441871876


In [238]:
# Greeks with dividend
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,underlying,strike,ttm,rf,np.array([div]),np.array([divtimes]),ivol,N,True)
print("Greeks for call:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("delta dividend : {}".format(ddiv))

Greeks for call:
Deta:0.5350316590583626
gamma:0.021277169572075927
Vega: 19.575283596213655
theta: -24.293425333044816
rho: 6.867783857751331
delta dividend : -0.11549740557335042


In [239]:
delta,gamma,vega,theta,rho,ddiv = fd_greeks_bt(0.2,0.01,0.01,0.01,0.01,underlying,strike,ttm,rf,np.array([div]),np.array([1]),ivol,2,False)
print("Greeks for Put:")
print("Deta:{}".format(delta))
print("gamma:{}".format(gamma))
print("Vega: {}".format(vega))
print("theta: {}".format(theta))
print("rho: {}".format(rho))
print("delta dividend : {}".format(ddiv))

Greeks for Put:
Deta:-0.7157072182794533
gamma:1.4432899320127035e-13
Vega: 17.615344905813178
theta: -16.95242792152771
rho: -4.3195014178173174
delta dividend : 0.7256234304063636


## Problem2

In [2]:
port_options = pd.read_csv("Problem2.csv")
daily_prices = pd.read_csv("DailyPrices.csv")

In [3]:
port_options['ExpirationDate']=port_options['ExpirationDate'].astype("datetime64[ns]")
port_options['OptionType'] = port_options['OptionType'].apply(lambda x: bool(x=="Call") if pd.notna(x) else x)

In [4]:
# Inputs
currdate = datetime(2023,3,3)
div_date = np.array([datetime(2023,3,15)])
rf = 0.0425
# ttm = (expiration-curr_date).days/365
N=50
# divtimes = int((div_date-curr_date).days/(expiration-curr_date).days *N)
divamts = np.array([1.00])
underlying =151.03

In [7]:
# Solve ivol using bt
def ivol_bt(underlying,strike,ttm,rf,divamts,divtimes,N,call,value,initvol):
    def sol_vol(x,underlying,strike,ttm,rf,divamts,divtimes,N,call,value):
        return bt_american_div(underlying,strike,ttm,rf,divamts,divtimes,x,N,call)-value
    vol = fsolve(sol_vol,initvol,args=(underlying,strike,ttm,rf,divamts,divtimes,N,call,value))
    return vol[0]

In [8]:
# Calculate implied vol
def row_fun_port(row,underlying,currdate,div_date,divamts,rf,N):
    ttm = ((row['ExpirationDate']-currdate).days)/365
    divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
    divtimes = np.array([divtime_cal(d) for d in div_date])
    vol = ivol_bt(underlying,row['Strike'],ttm,rf,divamts,divtimes,N,row['OptionType'],row['CurrentPrice'],0.2)
    return vol

In [9]:
port_options['Implied Vol'] = port_options[port_options['Type']=='Option'].apply(row_fun_port,args=(underlying,currdate,div_date,divamts,rf,N),axis=1)

In [10]:
# Calculate the value for each portfolio
def portvalue(row,underlying,currdate,div_date,divamts,rf,N,daysahead):
    # # Create a dataframe to store the values
    # port_value = pd.DataFrame(portdf['Portfolio']).copy()
    # port_value['value'] = np.NaN
    # # for each asset calculate the value using bt
    # Calculate the value using the currentdate moving ahead
    currdate += timedelta(days=daysahead)
    if row['Type']=='Stock':
            return underlying*row['Holding']
    else:
            ttm = ((row['ExpirationDate']-currdate).days)/365
            divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
            divtimes = np.array([divtime_cal(d) for d in div_date])
            value = bt_american_div(underlying,row['Strike'],ttm,rf,divamts,divtimes,row['Implied Vol'],N,row['OptionType'])
            return value * row['Holding']
    # # group assets by portfolio
    # port_result = port_value.groupby(['Portfolio']).sum()
    # return port_result

In [11]:
# Calculate the log return of AAPL
aapl_log = np.log(daily_prices['AAPL']/daily_prices['AAPL'].shift(1))
# Demean the series
aapl_h = aapl_log-aapl_log.mean()

In [12]:
# Simulate the price using normal
def normal_sim(r,ndays,p0,nsim=1000,seed=10):
    sigma = r.std()
    np.random.seed(seed)
    rsim = np.random.normal(0,sigma,(nsim,ndays))
    rsim_cum = np.sum(rsim,axis=1)
    psim = np.zeros(nsim)
    for i in range(nsim):
        psim[i]=p0*exp(rsim_cum[i])
    return psim

In [13]:
# Calculate delta for each asset
def asset_delta(row,underlying,currdate,div_date,divamts,rf,N,d):
    if row['Type']=='Stock':
        return 1
    else:
        ttm = ((row['ExpirationDate']-currdate).days)/365
        divtime_cal = lambda x: int((x- currdate).days/(row['ExpirationDate']-currdate).days *N)
        divtimes = np.array([divtime_cal(d) for d in div_date])
        delta = (bt_american_div(underlying+d,row['Strike'],ttm,rf,divamts,divtimes,row['Implied Vol'],N,row['OptionType'])- bt_american_div(underlying,row['Strike'],ttm,rf,divamts,divtimes,row['Implied Vol'],N,row['OptionType']))/d
        return delta

In [14]:
port_options['delta'] = port_options.apply(asset_delta,args=(underlying,currdate,div_date,divamts,rf,N,0.2),axis=1)

In [15]:
# Delta*Holding
port_options['delta_h'] = port_options['delta']*port_options['Holding']
port_options['pv'] = port_options['CurrentPrice']*port_options['Holding']

In [16]:
# Simulate the current prices
price_sim = normal_sim(aapl_h,10,151.03,nsim=1000,seed=10)

In [17]:
# Calculate the PL for each simulation
# The current value of portfolios
port_curr = port_options.apply(portvalue,args=(151.03,currdate,div_date,divamts,rf,N,0),axis=1)
pl_list = []
for i in range(len(price_sim)):
    pl = port_options.apply(portvalue,args=(price_sim[i],currdate,div_date,divamts,rf,N,10),axis=1) - port_curr
    pl_list.append(pl)
pl_sim = pd.concat(pl_list,axis=1)
pl_sim.set_index(port_options['Portfolio'],inplace=True)
portpl_sim = pl_sim.groupby(level=0).sum().T

In [18]:
# Calculate the mean
port_mean = portpl_sim.mean(axis=0)
stat_mean = pd.DataFrame(port_mean,columns=['Mean'])

In [19]:
print(port_mean)

Portfolio
Call            0.952987
CallSpread      0.000725
CoveredCall    -0.479103
ProtectedPut    1.334180
Put             0.948173
PutSpread       0.373035
Stock           0.548978
Straddle        1.901160
SynLong         0.004814
dtype: float64


In [26]:
#calculate VaR and ES
def delta_var_es(row):
    r_sim = np.random.normal(0,row['scaler'],10000)
    var = VaR(r_sim)*row['pv']
    es = ES(r_sim)*row['pv']
    return var,es

In [ ]:
sigma = aapl_h.var()
port_r_cal = port_options[['Portfolio','pv','delta_h']].groupby('Portfolio').sum()
port_r_cal['dr'] = underlying/port_r_cal['pv']*port_r_cal['delta_h']
port_r_cal['scaler']= np.sqrt(port_r_cal['dr']*sigma*port_r_cal['dr'])
port_r_cal['VaR'] = port_r_cal.apply(lambda x : delta_var_es(x)[0],axis=1)
port_r_cal['ES'] = port_r_cal.apply(lambda x : delta_var_es(x)[1],axis=1)

In [31]:
# Mean ES and Var
stat = pd.concat([stat_mean,port_r_cal[['VaR','ES']]],axis=1)

In [32]:
stat.to_csv('portstat.csv')

                  pv   delta_h         dr    scaler       VaR        ES
Portfolio                                                              
Call            6.80  0.594226  13.197943  0.295978  3.280267  4.035524
CallSpread      4.59  0.319230  10.503989  0.235563  1.795791  2.234965
CoveredCall   146.98  0.623904   0.641095  0.014377  3.485058  4.295819
ProtectedPut  154.04  0.699466   0.685798  0.015380  3.868369  4.981210
Put             4.85 -0.430455 -13.404465  0.300609  2.372024  3.015266
PutSpread       3.01 -0.221534 -11.115722  0.249282  1.202810  1.512874
Stock         151.03  1.000000   1.000000  0.022426  5.565147  7.048000
Straddle       11.65  0.163771   2.123120  0.047613  0.914737  1.143212
SynLong         1.95  1.024682  79.362905  1.779796  5.727635  7.108029


## Problem3

In [172]:
# Read in data
F_momentum_data = pd.read_csv("F-F_Momentum_Factor_daily.csv")
F_3factor_data = pd.read_csv("F-F_Research_Data_Factors_daily.csv")

In [173]:
# Clean and prepare the data
F_momentum_data['Date']=F_momentum_data['Date'].astype("str")
F_3factor_data['Date']=F_3factor_data['Date'].astype("str")
F_momentum_data['Date']=F_momentum_data['Date'].apply(lambda x: datetime.strptime(x,"%Y%m%d"))
F_3factor_data['Date']=F_3factor_data['Date'].apply(lambda x: datetime.strptime(x,"%Y%m%d"))

In [174]:
F_momentum_data.set_index('Date',inplace=True)
F_3factor_data.set_index('Date',inplace=True)

In [176]:
F_4factor_data=pd.concat([F_momentum_data,F_3factor_data],axis=1)
# Convert to decimal not percentage
F_4factor_data = F_4factor_data/100

In [190]:
print(F_4factor_data)

            Mom     Mkt-RF     SMB     HML       RF
Date                                               
1926-07-01     NaN  0.0010 -0.0025 -0.0027  0.00009
1926-07-02     NaN  0.0045 -0.0033 -0.0006  0.00009
1926-07-06     NaN  0.0017  0.0030 -0.0039  0.00009
1926-07-07     NaN  0.0009 -0.0058  0.0002  0.00009
1926-07-08     NaN  0.0021 -0.0038  0.0019  0.00009
...            ...     ...     ...     ...      ...
2023-01-25  0.0014  0.0000 -0.0004  0.0065  0.00017
2023-01-26 -0.0123  0.0108 -0.0058  0.0001  0.00017
2023-01-27 -0.0246  0.0036  0.0062 -0.0116  0.00017
2023-01-30  0.0136 -0.0138 -0.0010  0.0072  0.00017
2023-01-31 -0.0070  0.0157  0.0099 -0.0006  0.00017

[25419 rows x 5 columns]


In [178]:
dailyprices = pd.read_csv("DailyPrices.csv")

In [185]:
dailyprices['Date']=dailyprices['Date'].astype("datetime64[ns]")

In [186]:
tickers = ['AAPL','META','UNH','MA',
           'MSFT','NVDA','HD','PFE',
           'AMZN','BRK-B','PG','XOM',
           'TSLA','JPM','V','DIS',
           'GOOGL','JNJ','BAC','CSCO']

In [187]:
stock_r =[]
for t in tickers:
    r = np.log(dailyprices[t]/dailyprices[t].shift(1))
    stock_r.append(r)
stocks = pd.concat(stock_r,axis=1)
stocks.set_index(dailyprices['Date'],inplace=True)
stocks.dropna(inplace=True)

In [189]:
# Get the 1y data
F_4factor_1y = F_4factor_data.loc['2022-02-15':'2023-01-31']
stocks_1y = stocks.loc['2022-02-15':'2023-01-31']

In [192]:
# The X array for model
r_mkt = F_4factor_1y['Mkt-RF'].values.reshape(-1,1)
smb = F_4factor_1y['SMB'].values.reshape(-1,1)
hml = F_4factor_1y['HML'].values.reshape(-1,1)
umd = F_4factor_1y['Mom   '].values.reshape(-1,1)
X = np.concatenate((r_mkt,smb,hml,umd),axis=1)

In [193]:
# Fit the model using Linear regression
model=LinearRegression()
para_dict = {}
for t in tickers:
    # r_s-r_f
    r_s= (stocks_1y[t]- F_4factor_1y['RF']).values.reshape(-1,1)
    model.fit(X,r_s)
    para_dict[t]=(model.coef_[0],model.intercept_[0])

In [195]:
# 10Y facotr returns
F_4factor_10y = F_4factor_data['2013-01-31':'2023-01-31']

In [161]:
print(F_4factor_10y)

            Mom     Mkt-RF     SMB     HML       RF
Date                                               
2013-01-31 -0.0037 -0.0008  0.0069  0.0017  0.00000
2013-02-01  0.0031  0.0099  0.0007  0.0024  0.00000
2013-02-04  0.0000 -0.0119 -0.0014 -0.0017  0.00000
2013-02-05  0.0013  0.0107 -0.0008  0.0014  0.00000
2013-02-06 -0.0032  0.0015  0.0028  0.0014  0.00000
...            ...     ...     ...     ...      ...
2023-01-25  0.0014  0.0000 -0.0004  0.0065  0.00017
2023-01-26 -0.0123  0.0108 -0.0058  0.0001  0.00017
2023-01-27 -0.0246  0.0036  0.0062 -0.0116  0.00017
2023-01-30  0.0136 -0.0138 -0.0010  0.0072  0.00017
2023-01-31 -0.0070  0.0157  0.0099 -0.0006  0.00017

[2518 rows x 5 columns]


In [213]:
# er for stocks
er = []
for t in tickers:
    s_r =para_dict[t][1]+para_dict[t][0][0]*F_4factor_10y['Mkt-RF']+ para_dict[t][0][1]*F_4factor_10y['SMB'] \
    + para_dict[t][0][2]*F_4factor_10y['HML']+para_dict[t][0][3]*F_4factor_10y['Mom   ']+F_4factor_10y['RF']
    s_r = s_r.mean()*250
    er.append(s_r)
er = np.array(er)

In [214]:
er

array([ 0.11960648, -0.12756437,  0.2388658 ,  0.19146119,  0.11812763,
        0.13925335,  0.08397227,  0.04881949, -0.12454428,  0.1198568 ,
        0.06483685,  0.48659818, -0.22771619,  0.0826194 ,  0.21120255,
       -0.19662501, -0.06854585,  0.11144937, -0.12760639,  0.11903391])

In [244]:
ER_sum = pd.DataFrame(er,index=tickers)

In [215]:
# corr and std for stocks
corr = stocks.corr().to_numpy()
std = stocks.std().to_numpy()*sqrt(250)
covar = np.diag(std)@ corr @np.diag(std)

In [221]:
# Calculate the optimal portfolio
def optimize_port(er,covar,rf):
    n = len(er)
    def sharpe_cal(w):
        r = w.T @ er.reshape(-1,1)
        std = sqrt(w.T @ covar @ w)
        sharpe = (r-rf)/std
        return -sharpe
    #Initial weights
    w0 = np.array([1/n]*n).reshape(-1,1)
    w_optmize = minimize(sharpe_cal,w0,
                         constraints=({'type':'ineq','fun': lambda x: x},{'type':'ineq','fun': lambda x: 1-x},{'type':'eq','fun': lambda x:x.sum()-1}))
    sharpe = -w_optmize.fun
    return w_optmize.x,sharpe

In [229]:
weights,sharpe = optimize_port(er,covar,0.0425)
weight_res = pd.DataFrame(weights,index=tickers,columns=['Weight'])
weight_res['Weight'] = weight_res['Weight'].apply(lambda x: round(x,4))

/var/folders/32/cdr8k22j5d31qndx08zm5j440000gn/T/ipykernel_12123/764910999.py:11: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  w_optmize = minimize(sharpe_cal,w0,


In [227]:
print("Maximum Sharpe ratio:{}".format(sharpe))

Maximum Sharpe ratio:1.3574956159710518


In [232]:
weight_res.to_csv('weights.csv')

In [245]:
ER_sum.to_csv('ER.csv')